# 🔄 Fine-Tune and Serve LLMs with Union.ai: A Hands-On Tutorial

<a target="_blank" href="https://colab.research.google.com/github/unionai-oss/bert-llm-classification-pipeline/blob/main/tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Welcome to this step-by-step tutorial on building a **Large Language Model (LLM) fine-tuning pipeline** using **Hugging Face Transformers** and **Union.ai’s AI workflow and inference platform**. In this tutorial, you’ll train a **BERT-based model for text classification**, serve it for inference, and track every step of your pipeline using **Union’s powerful MLOps capabilities**.  

This example might seem simple, but the **core concepts and tools** covered here apply to real-world **AI and machine learning (ML) projects** at any scale. By following along, you'll gain hands-on experience in:  

🔸 **Automating ML workflows** with Union.ai  
🔸 **Fine-tuning a transformer model** with Hugging Face  
🔸 **Deploying a model for inference** and tracking artifacts  
🔸 **Optimizing your pipeline** with caching and versioning  

## ✨ Why Use Union.ai?  

By just adding a few lines of code to your Python functions, you'll be able to create a reproducible ML pipeline, taking advantage of Union's features:

With just **a few lines of code**, you can transform your Python functions into **scalable, reproducible AI workflows**. Here’s what you get:  

- **🛠 Reproducible AI Workflows** – Ensure your pipeline runs in the same environment every time.  
- **📌 Versioning & Tracking** – Automatically track **code, models, and artifacts**.  
- **⚡ Faster Iterations with Data Caching** – Reuse previous results to speed up experiments.  
- **🖥 Declarative Infrastructure** – Define ML infrastructure **in code** without manual setup.  
- **📂 Artifact Management** – Keep track of model checkpoints and datasets seamlessly.  
- **📦 Containerized Execution** – Deploy models in a consistent environment with automatic **image building**.  
- **🧑‍💻 Local & Cloud Development** – Test locally before scaling up.  
- **🎭 Actors for Long Running Stateful Containers** – Run **Effceint batch inference** with persistent containers.  
- **…and much more!** 

## 📝 What You'll Build  

By the end of this tutorial, you'll have a **fully functional AI pipeline** that:  

1. **Downloads and processes a dataset** 📥  
2. **Fine-tunes a BERT model for classification** 🏋️‍♂️  
3. **Saves and versions the trained model** 💾  
4. **Deploys the model for real-time inference** 🚀  
5. **Tracks all artifacts and experiments** using Union.ai 📊  



## 🧰 Setup 


To get started, sign up for a **Union Serverless** account at [Union.ai](https://union.ai) by clicking the **"Get Started"** button. No credit card is required, and you'll receive **$30 in free credits** to begin experimenting. The signup process takes just a few minutes.  

Alternatively, if you have access to a **[Union BYOC Enterprise](https://www.union.ai/pricing)** account, you can log into your account.  

### 📦 Install Python Packages & Clone Repo

Packages can be installed in your local environment using the following command using your preferred package manager from the [requirements.txt](requirements.txt) file. For example `pip install -r requirements.txt`. 

to clone the repo, run the following command in your environment: `git clone https://github.com/unionai-oss/bert-llm-classification-pipeline`

If you're running this notebook in a Google Colab environment, you can install the packages and clone the GitHub repo directly in the notebook by running the following cell:


In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone https://github.com/unionai-oss/bert-llm-classification-pipeline
    %cd bert-llm-classification-pipeline
    !pip install -r requirements.txt

### 🔐 Authenticate
To use **Union.ai**, you'll need to authenticate your account. Follow the appropriate step based on your setup:  

##### 🔸 **Using Union BYOC Enterprise**  

If you're using a **[Union BYOC Enterprise](https://www.union.ai/pricing)** account, log in with the following command:  
```bash
union create login --host <union-host-url>
```

Replace <union-host-url> with your organization's Union instance URL.

##### 🔸 Using Union Serverless
If you're using [Union Serverless](https://www.union.ai/) , authenticate by running the command below:

Create an account for free at [Union.ai](https://union.ai) if you don't have one yet:
 

In [ ]:
# 🌟 Authenticate to union serverless
!union create login --serverless --auth device-flow

## 🧩 Create a Simple Workflow

Before we build our ML pipeline lets build a simple workflow to understand the basics of Union's workflow system.

`ImageSpec` - Allows you to specify the environment in which your task will run directly in your Python code. This includes the Python packages, CUDA version, and any additional environment setup you need. When a task is run, Union will automatically build a container image with the specified environment if it doens't already exsist and run the task in that container.

`Tasks` - Tasks are the building blocks of workflows. They allow you to define a unit of work and what infrastructure to us.

`Workflows` - A workflow is a collection of tasks that and defines data flow. Workflows can be run locally or in the cloud.

Both tasks workflows are strongly typed

Note: We could build our whole ML pipeline directly in the notebook like below if we wanted to!

In [ ]:
import flytekit as fl
import sys

image = fl.ImageSpec(
    name="notebook-example",
    packages=[
        "flytekit==1.15.0",
        "union==0.1.144",
    ],
    python_version=f"{sys.version_info.major}.{sys.version_info.minor}",
    builder="union",
)

@fl.task(container_image=image, 
         requests=fl.Resources(cpu="1", mem="1Gi"))
def hello_world(name: str) -> str:
    """Returns a greeting."""
    return f"Hello, {name}!"

@fl.workflow
def my_workflow(name: str ="union.ai") -> str:
    return hello_world(name=name)

In [ ]:
from union.remote import UnionRemote
serverless = UnionRemote()

In [ ]:
exe = serverless.execute(my_workflow, inputs={"name": "Flyte"})
exe

In [ ]:
exe.wait(poll_interval=1)
response = exe.outputs['o0']
response

## 🔀 BERT Fine-Tuning Pipeline  

In this section, we'll execute **tasks and workflows** defined in Python under the relevant folders.  

📂 Navigate to the `tasks` and `workflows` folders to explore the code. If you're following along in a **hosted Jupyter Notebook**, you can view the files by clicking the **folder icon** (usually on the left side of the screen).  

### 🛠 Workflow Overview  

We’ll create an **end-to-end machine learning pipeline** to train a **BERT model for text classification** using the **Iris dataset**. The workflow consists of the following steps:  

1. 📥 **Download & Preprocess Dataset**   
2. 🤖 **Download Pretrained BERT Model**   
3. 🏋️‍♂️ **Fine-tune BERT Model**   
4. 📊 **Evaluate Model Performance**   
5. 💾 **Save Model as an Artifact**  *(We’ll serve the model in the next section)*  
6. 🔍 **Run a Prediction on New Test Data**   

> **💡 Note:**  
> In more complex ML workflows, **data pipelines** are often separate from **model training pipelines**.  
> For simplicity, we'll combine them into a single workflow in this example.  

### 🔎 Explore the Code  

To view the workflow, navigate to the [`workflows/train_pipeline.py`](workflows/train_pipeline.py) file.  

- Look for the **`train_pipeline()`** function—this defines the full workflow.  
- The workflow **calls tasks** from the [`tasks`](tasks/) folder.  
- It also **builds a container image** using [`containers.py`](containers.py).  

Once you understand the structure, **run the workflow** and track your pipeline execution with **Union.ai**! 🚀  


In [ ]:
# 🌟 Run the bert training pipeline using LoRA, QLoRA or FFT
!union run --remote workflows/train_pipeline.py train_pipeline --epochs 3 --tuning_method lora 

Run the command above to run the workflow. This will automatically build the container image and run the tasks in the workflow. You can view the progress and logs in the Union UI. 

We'll go through each workflows and tasks in detail below. Tasks are defined in the `tasks` folder. Workflows are defined in the `workflows` folder.

These were already cloned in the setup step above. If you are running this in a local environment, make sure to clone the repo and install the requirements. 

In [ ]:
%%writefile workflows/train_pipeline.py

"""
This file contains the train_pipeline workflow that orchestrates the
training pipeline for BERT classification models
"""

from union import workflow

from tasks.data import download_dataset, visualize_data
from tasks.inference import predict_batch_sentiment
from tasks.model import download_model, evaluate_model, train_model


# ---------------------------
# train pipeline
# ---------------------------
@workflow
def train_pipeline(
    tuning_method: str = "lora",  # options: "full", "lora", "qlora"
    model_name: str = "distilbert-base-uncased",
    epochs: int = 3,
    extra_test_text: list[str] = [
        "This is a great movie!",
        "This is a bad movie!",
    ],
) -> None:

    train_dataset, val_dataset, test_dataset = download_dataset()
    saved_model_dir = download_model(model_name=model_name)

    visualize_data(
        train_dataset=train_dataset, val_dataset=val_dataset, test_dataset=test_dataset
    )

    trained_model_dir = train_model(
        tuning_method=tuning_method,
        model_dir=saved_model_dir,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        epochs=epochs,
    )

    evaluate_model(trained_model_dir=trained_model_dir, test_dataset=test_dataset)

    # Perform batch inference
    predict_batch_sentiment(trained_model_dir=trained_model_dir, texts=extra_test_text)

# Run model training pipeline:
#!union run --remote workflows/train_pipeline.py train_pipeline

In [ ]:
%%writefile tasks/data.py

"""
This module contains tasks for downloading the dataset and visualizing the data.
"""

from pathlib import Path

from datasets import Dataset
from flytekit.types.directory import FlyteDirectory
from flytekit.types.file import FlyteFile
from typing_extensions import Annotated
from union import Artifact, Deck, Resources, current_context, task

from containers import container_image

# Define Artifact Specifications
RawImdbDataset = Artifact(name="raw_imdb_dataset")
TrainImdbDataset = Artifact(name="train_imdb_dataset")
ValImdbDataset = Artifact(name="val_imdb_dataset")
TestImdbDataset = Artifact(name="test_imdb_dataset")


# ---------------------------
# download dataset
# ---------------------------
@task(
    container_image=container_image,
    cache=True,
    cache_version="1",
    requests=Resources(cpu="2", mem="2Gi"),
)
def download_dataset() -> tuple[
    Annotated[FlyteFile, TrainImdbDataset],
    Annotated[FlyteFile, ValImdbDataset],
    Annotated[FlyteFile, TestImdbDataset],
]:

    import pandas as pd
    from datasets import load_dataset
    from sklearn.model_selection import train_test_split

    # Load IMDB dataset
    dataset = load_dataset("imdb")
    train_df = dataset["train"].to_pandas()
    test_df = dataset["test"].to_pandas()

    # Split training set into train and validation sets
    train_df, val_df = train_test_split(
        train_df, test_size=0.2, stratify=train_df["label"], random_state=42
    )

    working_dir = Path(current_context().working_directory)
    data_dir = working_dir / "data"
    data_dir.mkdir(parents=True, exist_ok=True)

    # Save datasets as CSV files
    train_path = data_dir / "train.csv"
    val_path = data_dir / "val.csv"
    test_path = data_dir / "test.csv"

    train_df.to_csv(train_path, index=False)
    val_df.to_csv(val_path, index=False)
    test_df.to_csv(test_path, index=False)

    return (
        TrainImdbDataset.create_from(train_path),
        ValImdbDataset.create_from(val_path),
        TestImdbDataset.create_from(test_path),
    )


# ---------------------------
# visualize data
# ---------------------------
@task(
    container_image=container_image,
    enable_deck=True,
    requests=Resources(cpu="2", mem="2Gi"),
)
def visualize_data(
    train_dataset: FlyteFile, val_dataset: FlyteFile, test_dataset: FlyteFile
):
    import base64
    from textwrap import dedent

    import matplotlib.pyplot as plt
    import pandas as pd

    ctx = current_context()

    # Load datasets from CSV files
    train_df = pd.read_csv(train_dataset.download())
    val_df = pd.read_csv(val_dataset.download())
    test_df = pd.read_csv(test_dataset.download())

    # Create the deck for visualization
    deck = Deck("Dataset Analysis")

    # Sample reviews from the datasets
    train_positive_review = train_df[train_df["label"] == 1].iloc[0]["text"]
    train_negative_review = train_df[train_df["label"] == 0].iloc[0]["text"]
    val_positive_review = val_df[val_df["label"] == 1].iloc[0]["text"]
    val_negative_review = val_df[val_df["label"] == 0].iloc[0]["text"]
    test_positive_review = test_df[test_df["label"] == 1].iloc[0]["text"]
    test_negative_review = test_df[test_df["label"] == 0].iloc[0]["text"]

    # Visualization helper
    def plot_label_distribution(df, title, color, output_path):
        plt.figure(figsize=(10, 5))
        df["label"].value_counts().plot(kind="bar", color=color)
        plt.title(title)
        plt.xlabel("Label")
        plt.ylabel("Count")
        plt.tight_layout()
        plt.savefig(output_path)
        plt.close()

    # Plot label distributions
    plot_label_distribution(
        train_df,
        "Train Data Label Distribution",
        "skyblue",
        "/tmp/train_label_distribution.png",
    )
    plot_label_distribution(
        val_df,
        "Validation Data Label Distribution",
        "orange",
        "/tmp/val_label_distribution.png",
    )
    plot_label_distribution(
        test_df,
        "Test Data Label Distribution",
        "lightgreen",
        "/tmp/test_label_distribution.png",
    )

    # Convert images to base64 for embedding
    def image_to_base64(image_path):
        with open(image_path, "rb") as img_file:
            return base64.b64encode(img_file.read()).decode("utf-8")

    train_image_base64 = image_to_base64("/tmp/train_label_distribution.png")
    val_image_base64 = image_to_base64("/tmp/val_label_distribution.png")
    test_image_base64 = image_to_base64("/tmp/test_label_distribution.png")

    # Create HTML report
    html_report = dedent(
        f"""
    <div style="font-family: Arial, sans-serif; line-height: 1.6;">
        <h2 style="color: #2C3E50;">Dataset Analysis</h2>

        <h3 style="color: #2980B9;">Training Data Summary</h3>
        <img src="data:image/png;base64,{train_image_base64}" alt="Train Data Label Distribution" width="600">
        Shape: {train_df.shape} <br>
        Label Distribution: {train_df['label'].value_counts()} <br>
        <p><strong>Positive Review:</strong> {train_positive_review}</p>
        <p><strong>Negative Review:</strong> {train_negative_review}</p>

        <h3 style="color: #2980B9;">Validation Data Summary</h3>
        <img src="data:image/png;base64,{val_image_base64}" alt="Validation Data Label Distribution" width="600">
        Shape: {val_df.shape} <br>
        Label Distribution: {val_df['label'].value_counts()} <br>
        <p><strong>Positive Review:</strong> {val_positive_review}</p>
        <p><strong>Negative Review:</strong> {val_negative_review}</p>

        <h3 style="color: #2980B9;">Test Data Summary</h3>
        <img src="data:image/png;base64,{test_image_base64}" alt="Test Data Label Distribution" width="600">
        Shape: {test_df.shape} <br>
        Label Distribution: {test_df['label'].value_counts()} <br>
        <p><strong>Positive Review:</strong> {test_positive_review}</p>
        <p><strong>Negative Review:</strong> {test_negative_review}</p>
    </div>
    """
    )

    # Append HTML content to the deck
    deck.append(html_report)

    # Insert the deck into the context
    ctx.decks.insert(0, deck)


In [ ]:
%%writefile tasks/model.py
"""
This file contains the tasks that are used to download, train and evaluate the model.
"""

from pathlib import Path
from flytekit.types.directory import FlyteDirectory
from flytekit.types.file import FlyteFile
from typing_extensions import Annotated
from union import Artifact, Deck, Resources, current_context, task
from containers import container_image

# Define Artifact Specifications
FineTunedImdbModel = Artifact(name="fine_tuned_Imdb_model")

# ---------------------------
# download model
# ---------------------------
@task(
    container_image=container_image,
    cache=True,
    cache_version="1",
    requests=Resources(cpu="2", mem="2Gi"),
)
def download_model(model_name: str) -> FlyteDirectory:
    from transformers import AutoModelForSequenceClassification, AutoTokenizer

    working_dir = Path(current_context().working_directory)
    saved_model_dir = working_dir / "saved_model"
    saved_model_dir.mkdir(parents=True, exist_ok=True)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        device_map="cpu",
        torch_dtype="auto",
        trust_remote_code=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model.save_pretrained(saved_model_dir)
    tokenizer.save_pretrained(saved_model_dir)

    return FlyteDirectory(saved_model_dir)


# ---------------------------
# full/lora/qlora fine-tune model
# ---------------------------
@task(
    container_image=container_image,
    requests=Resources(cpu="4", mem="12Gi", gpu="1"),
)
def train_model(
    model_dir: FlyteDirectory,
    train_dataset: FlyteFile,
    val_dataset: FlyteFile,
    epochs: int = 3,
    tuning_method: str = "lora",  # options: "full", "lora", "qlora"
    lora_r: int = 8,
    lora_alpha: int = 16,
    lora_dropout: float = 0.1,
) -> Annotated[FlyteDirectory, FineTunedImdbModel]:
    import pandas as pd
    import torch
    from datasets import Dataset
    from transformers import (
        AutoModelForSequenceClassification,
        AutoTokenizer,
        Trainer,
        TrainingArguments,
    )
    from peft import prepare_model_for_kbit_training

    local_model_dir = model_dir.download()
    train_df = pd.read_csv(train_dataset.download()).sample(n=5000, random_state=42)
    val_df = pd.read_csv(val_dataset.download()).sample(n=1000, random_state=42)

    train_dataset_hf = Dataset.from_pandas(train_df)
    val_dataset_hf = Dataset.from_pandas(val_df)

    tokenizer = AutoTokenizer.from_pretrained(local_model_dir)

    def tokenizer_function(example):
        return tokenizer(example["text"], padding="max_length", truncation=True)

    tokenized_train = train_dataset_hf.map(tokenizer_function)
    tokenized_val = val_dataset_hf.map(tokenizer_function)

    if tuning_method == "qlora":
        from transformers import BitsAndBytesConfig
        # from peft.utils import prepare_model_for_kbit_training
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
            llm_int8_skip_modules=["classifier", "pre_classifier"],
        )
        model = AutoModelForSequenceClassification.from_pretrained(
            local_model_dir,
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
            # device_map="auto", # use this for most models if implemented
        )

    else:
        model = AutoModelForSequenceClassification.from_pretrained(local_model_dir)

    if tuning_method in {"lora", "qlora"}:
        from peft import get_peft_model, LoraConfig, TaskType
        lora_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            r=lora_r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout,
            target_modules=["q_lin", "k_lin", "v_lin"],
        )

        # if tuning_method == "qlora":
        #     # Ensure LoRA target modules are in float dtype, not int4
        #     for name, module in model.named_modules():
        #         if any(target in name for target in ["q_lin", "k_lin", "v_lin"]):
        #             module.to(torch.bfloat16)

        model = get_peft_model(model, lora_config)

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
    )

    trainer.train()

    # Merge LoRA weights into base model
    if tuning_method in {"lora", "qlora"}:
        model = model.merge_and_unload()

    output_dir = Path(current_context().working_directory) / "trained_model"
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    #TODO: Save traning type (lora, qlora, full) as artifacts
    return FineTunedImdbModel.create_from(output_dir)



# ---------------------------
# evaluate model
# ---------------------------
@task(
    container_image=container_image,
    enable_deck=True,
    requests=Resources(cpu="2", mem="12Gi", gpu="1"),
)
def evaluate_model(trained_model_dir: FlyteDirectory, test_dataset: FlyteFile) -> dict:
    import numpy as np
    import pandas as pd
    from datasets import Dataset
    from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
    from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
    from sklearn.metrics import confusion_matrix, roc_curve, auc
    import seaborn as sns
    import matplotlib.pyplot as plt
    import base64
    from union import current_context
    from union import Deck
    from textwrap import dedent

    # Download model locally
    local_model_dir = trained_model_dir.download()
    ctx = current_context()

    # Load model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(
        local_model_dir,
        torch_dtype="auto",
        load_in_4bit=False,  # Important: for evaluation, avoid loading in quantized 4-bit unless you really want to
    )
    tokenizer = AutoTokenizer.from_pretrained(local_model_dir)

    # Load and prepare the test dataset
    test_df = pd.read_csv(test_dataset.download()).sample(n=1000, random_state=42)

    # Use a pipeline for evaluation (bypasses Trainer and works for quantized models)
    nlp_pipeline = pipeline(
        "text-classification",
        model=model,
        tokenizer=tokenizer,
        # device=0 if torch.cuda.is_available() else -1,  # auto-select device
        truncation=True,
        padding=True,
    )

    # Perform batch inference
    predictions = nlp_pipeline(test_df["text"].tolist(), batch_size=8)

    # Extract predicted labels
    pred_labels = [int(p["label"].split("_")[-1]) if "label" in p else 0 for p in predictions]
    true_labels = test_df["label"].tolist()

    # Calculate metrics
    metrics = {
        "accuracy": accuracy_score(true_labels, pred_labels),
        "f1": f1_score(true_labels, pred_labels, average="weighted"),
        "precision": precision_score(true_labels, pred_labels, average="weighted"),
        "recall": recall_score(true_labels, pred_labels, average="weighted"),
        # "conf_matrix": confusion_matrix(true_labels, pred_labels)
    }

    # create visualization deck
    deck = Deck("Model Evaluation")

    # Generate Confusion Matrix
    cm = confusion_matrix(true_labels, pred_labels)
    cm_path = f"/tmp/confusion_matrix.png"
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=sorted(set(true_labels)), yticklabels=sorted(set(true_labels)))
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.savefig(cm_path)
    plt.close()
    
    # # Generate ROC Curve
    # if len(set(true_labels)) == 2:  # Only for binary classification
    #     fpr, tpr, _ = roc_curve(true_labels, probs[:, 1])
    #     roc_auc = auc(fpr, tpr)
    #     roc_path = f"tmp/roc_curve.png"
    #     plt.figure(figsize=(8, 6))
    #     plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (area = {roc_auc:.2f}')
    #     plt.plot([0, 1], [0, 1], color='grey', linestyle='--')
    #     plt.xlim([0.0, 1.0])
    #     plt.ylim([0.0, 1.05])
    #     plt.xlabel('False Positive Rate')
    #     plt.ylabel('True Positive Rate')
    #     plt.title('Receiver Operating Characteristic')
    #     plt.legend(loc="lower right")
    #     plt.savefig(roc_path)
    #     plt.close()
    # else:
    #     roc_path = None
    
    # Convert images to base64 for embedding
    def image_to_base64(image_path):
        with open(image_path, "rb") as img_file:
            return base64.b64encode(img_file.read()).decode("utf-8")
        
    cm_image_base64 = image_to_base64(cm_path)
    # roc_image_base64 = image_to_base64(roc_path) if roc_path else None

    # Create HTML report
    html_report = dedent(
        f"""
    <div style="font-family: Arial, sans-serif; line-height: 1.6;">
        <h2 style="color: #2C3E50;">Model Evaluation</h2>

        <h3 style="color: #2980B9;">Confusion Matrix</h3>
        <img src="data:image/png;base64,{cm_image_base64}" alt="Confusion Matrix" width="600">
        <h3 style="color: #2980B9;">Model Metrics</h3>
        <pre>{metrics}</pre>
        
    </div>
        """)

     # Append HTML content to the deck
    deck.append(html_report)
    # Insert the deck into the context
    ctx.decks.insert(0, deck)


    return metrics


# 🚀 Serving the Fine-Tuned BERT model:

### Live App Serving (Beta)

Union.ai provides a **simple way to serve your models as a live app**, making it easy to interact with your trained model.  

In this example, we'll deploy the model using **Streamlit**, which provides a **simple web interface** for running predictions.  


📂 Check out the following files for the model-serving code:  
- [`app.py`](app.py) – Defines the **Streamlit-based UI** for interacting with the model.  
- [`main.py`](main.py) – Handles **loading the model** and serving it via Union.ai.  

Deploy the model by running the following command:

In [ ]:
!union deploy apps app.py bert-sentiment-analysis

Check the Union platform `Apps` tab to see the status of all apps!

Once the app is live, experiment with different inputs and see how your fine-tuned BERT model performs! 🚀

### Batch Serving

Union.ai also provides a way to serve your models in batch mode. This is useful when you have a large number of predictions to make and you want to do them all at once.

In [ ]:
!union register workflows/batch_inference.py

In [ ]:
from union.remote import UnionRemote
# Create a remote connection
remote = UnionRemote()

In [ ]:
def predict_with_container(data):

    inputs = {"texts": data}

    workflow = remote.fetch_workflow(name="workflows.batch_inference.batch_inference_workflow")
    execution = remote.execute(workflow, inputs=inputs, wait=True) # wait=True will block until the execution is complete

    # print(execution.outputs)

    return execution.outputs["o0"]

In [ ]:
print(predict_with_container(["I love this movie",
                               "I hate this movie"]
                               ))

### ⚡ Faster batch serving with Union Actors

Union [Actors](https://docs.union.ai/serverless/user-guide/core-concepts/actors/#actors) dramatically reduce the cost of cold starts by maintaining long-running stateful environments that stay ready for use until a defined time-to-live (TTL). This persistent setup eliminates redundant initialization and unlocks several key benefits. This can be especially useful for AI pipelines that benefit from long-running environments, such as large containers, serving models,

In [ ]:
def predict_with_actor(data):

    inputs = {"texts": data}

    workflow = remote.fetch_workflow(name="workflows.batch_inference.actor_batch_inference_workflow")
    execution = remote.execute(workflow, inputs=inputs, wait=True) # wait=True will block until the execution is complete

    # print(execution.outputs)

    return execution.outputs['o0']

In [ ]:
print(predict_with_actor(["I love this movie",
                               "I hate this movie"]
                               ))

In [ ]:
print(predict_with_actor(["I love this movie",
                               "I hate this movie"]
                               ))

In [ ]:
print(predict_with_actor(["I love this movie",
                               "I hate this movie"]
                               ))